In [85]:
import numpy as np
import glob
import os
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from pyproj import CRS
import xarray as xr
import rioxarray  
from datetime import datetime, timedelta
from rasterio.enums import Resampling
from rio_cogeo.cogeo import cog_translate
from rio_cogeo.profiles import cog_profiles
import tempfile

In [86]:
all_files= glob.glob("labelled_plumes2/*/*/*.nc")

In [87]:
plume_metadata = {
    "BV1": {
        "2019-04-07": {
            "Plume_id": "BV1-1",
            "Plume_Lat": "26.298" ,
            "Plume_Lon": "-104.530" ,
            "state": "Durango",
            "country": "Mexico"
        },
        "2019-05-21": {
            "Plume_id": "BV1-2",
            "Plume_Lat": "26.298", 
            "Plume_Lon": "-104.530",
            "state": "Durango",
            "country": "Mexico"
        }
    },
    "BV2": {
        "2019-04-09": {
            "Plume_id": "BV2-1",
            "Plume_Lat": "26.086", 
            "Plume_Lon": "-104.317",
            "state": "Durango",
            "country": "Mexico"
        },
        "2019-04-10": {
            "Plume_id": "BV2-2",
            "Plume_Lat": "26.086", 
            "Plume_Lon": "-104.317" ,
            "state": "Durango",
            "country": "Mexico"
        },
        "2019-04-18": {
            "Plume_id": "BV2-3",
            "Plume_Lat": "26.086", 
            "Plume_Lon": "-104.317" ,
            "state": "Durango",
            "country": "Mexico"
        },
        "2019-05-11": {
            "Plume_id": "BV2-4", 
            "Plume_Lat": "26.086", 
            "Plume_Lon": "-104.317" ,
            "state": "Durango",
            "country": "Mexico"
        },
        "2019-05-12": {
            "Plume_id": "BV2-5",
            "Plume_Lat": "26.086", 
            "Plume_Lon": "-104.317" ,
            "state": "Durango",
            "country": "Mexico"
        }
    },
    "BV4": {
        "2019-05-16": {
            "Plume_id": "BV4-1",
            "Plume_Lat": "26.086", 
            "Plume_Lon": "-104.356" ,
            "state": "Durango",
            "country": "Mexico"
        },
        "2019-05-22": {
            "Plume_id": "BV4-2",
            "Plume_Lat": "26.086", 
            "Plume_Lon": "-104.356" ,
            "state": "Durango",
            "country": "Mexico"
        },
        "2019-05-23": {
            "Plume_id": "BV4-3",
            "Plume_Lat": "26.086", 
            "Plume_Lon": "-104.356" ,
            "state": "Durango",
            "country": "Mexico"
        },
        "2019-05-24": {
            "Plume_id": "BV4-4",
            "Plume_Lat": "26.086", 
            "Plume_Lon": "-104.356" ,
            "state": "Durango",
            "country": "Mexico"
        }
    },
    "IN-1": {
        "2020-06-15": {
            "Plume_id": "IN-1",
            "Plume_Lat": "40.650", 
            "Plume_Lon": "-87.000" ,
            "state": "Indiana",
            "country": "USA"
        }
    },
    "IN-2": {
    "2020-06-15": {
        "Plume_id": "IN-2",
        "Plume_Lat": "40.650", 
        "Plume_Lon": "-87.000" ,
            "state": "Indiana",
            "country": "USA"
    }
},
    "Permian": {
        "2023-07-26": {
            "Plume_id": "PB-1",
            "Plume_Lat": "33.645", 
            "Plume_Lon": "-104.779" ,
            "state": "Texas",
            "country": "USA"
        }
    }
}


In [93]:
src_crs = CRS.from_wkt('PROJCS[\"unknown\",GEOGCS[\"unknown\",DATUM[\"unknown\",SPHEROID[\"GRS 1980\",6378137,298.2572221]],PRIMEM[\"Greenwich\",0,AUTHORITY[\"EPSG\",\"8901\"]],UNIT[\"degree\",0.0174532925199433,AUTHORITY[\"EPSG\",\"9122\"]]],PROJECTION[\"Geostationary_Satellite\"],PARAMETER[\"central_meridian\",-75],PARAMETER[\"satellite_height\",35786023],PARAMETER[\"false_easting\",0],PARAMETER[\"false_northing\",0],UNIT[\"metre\",1,AUTHORITY[\"EPSG\",\"9001\"]],AXIS[\"Easting\",EAST],AXIS[\"Northing\",NORTH],EXTENSION[\"PROJ4\",\"+proj=geos +a=6378137.0 +rf=298.2572221 +lon_0=-75.0 +lat_0=0.0 +h=35786023.0 +x_0=0 +y_0=0 +units=m +sweep=x +no_defs\"]]')
dst_crs = CRS.from_epsg(4326) 
for file in all_files:

    site_name= file.split('/')[-3].split("_")[0]
    original_time = file.split('/')[-1].split('.')[0]
    parsed_time = datetime.strptime(original_time, '%Y-%m-%dT%Hh%M')
    formatted_time = parsed_time.strftime('%Y-%m-%dT%H:%M:%S') + 'Z'

    try:
        plume_id = plume_metadata[site_name][formatted_time.split('T')[0]]["Plume_id"]
        state = plume_metadata[site_name][formatted_time.split('T')[0]]["state"]
        country = plume_metadata[site_name][formatted_time.split('T')[0]]["country"]
    except KeyError:
        print(f"KeyError for file", file)
        
    ds = xr.open_dataset(file)
    all_zero = np.all( ds['CH4_plume_mask'] == 0)
    if ~(all_zero):
            
        data_array = ds['Rad'] * ds['CH4_plume_mask']

        # Identify the bounding box where CH4_plume_mask is 1
        y, x = np.where(ds['CH4_plume_mask'].values == 1)
        latitudes = ds['y'].values[y]
        longitudes = ds['x'].values[x]

        # Calculate the bounding box coordinates
        min_lon, max_lon = longitudes.min(), longitudes.max()
        min_lat, max_lat = latitudes.min(), latitudes.max()

        # Assign the spatial coordinates (for rioxarray to work properly)
        data_array = data_array.rio.write_crs(src_crs)  # Update CRS as needed
        
        # Crop the data array to the bounding box
        data_array = data_array.rio.clip_box(
            minx=min_lon,
            miny=min_lat,
            maxx=max_lon,
            maxy=max_lat
        )

        data_array.rio.write_nodata(0, inplace=True)
        
        data_array.rio.write_crs(src_crs.to_string(), inplace=True)
        reprojected_data = data_array.rio.reproject(dst_crs.to_string())
        reprojected_data.rio.set_spatial_dims("x", "y", inplace=True)
        
        reprojected_data.where(reprojected_data != 0,-9999 )
        reprojected_data.rio.write_nodata(0, inplace=True)
        if '_FillValue' in reprojected_data.attrs:
            del reprojected_data.attrs['_FillValue']

    
        if site_name in ["IN-1", "IN-2"]:
            site_name="Indiana"
    
        filename = f"GOES-CH4_{country}_{state}_{site_name}_{plume_id}_{formatted_time}"
        print(filename)
        output_file = f"output3/{filename}.tif"
        reprojected_data.rio.to_raster(output_file,  compress='deflate', nodata=0, driver="COG")


GOES-CH4_USA_Texas_Permian_PB-1_2023-07-26T17:11:00Z
GOES-CH4_USA_Texas_Permian_PB-1_2023-07-26T17:01:00Z
GOES-CH4_USA_Texas_Permian_PB-1_2023-07-26T17:21:00Z
GOES-CH4_USA_Texas_Permian_PB-1_2023-07-26T17:31:00Z
GOES-CH4_USA_Texas_Permian_PB-1_2023-07-26T17:41:00Z
GOES-CH4_USA_Texas_Permian_PB-1_2023-07-26T17:51:00Z
GOES-CH4_USA_Texas_Permian_PB-1_2023-07-26T16:41:00Z
GOES-CH4_USA_Texas_Permian_PB-1_2023-07-26T16:51:00Z
GOES-CH4_USA_Texas_Permian_PB-1_2023-07-26T16:46:00Z
GOES-CH4_USA_Texas_Permian_PB-1_2023-07-26T16:56:00Z
GOES-CH4_USA_Texas_Permian_PB-1_2023-07-26T16:36:00Z
GOES-CH4_USA_Texas_Permian_PB-1_2023-07-26T18:01:00Z
GOES-CH4_USA_Texas_Permian_PB-1_2023-07-26T17:46:00Z
GOES-CH4_USA_Texas_Permian_PB-1_2023-07-26T17:56:00Z
GOES-CH4_USA_Texas_Permian_PB-1_2023-07-26T17:26:00Z
GOES-CH4_USA_Texas_Permian_PB-1_2023-07-26T17:36:00Z
GOES-CH4_USA_Texas_Permian_PB-1_2023-07-26T17:16:00Z
GOES-CH4_USA_Texas_Permian_PB-1_2023-07-26T17:06:00Z
GOES-CH4_USA_Indiana_Indiana_IN-1_2020-06-15T1

In [94]:
# Add overviews
COG_PROFILE = {"driver": "COG", "compress": "DEFLATE"}
OVERVIEW_LEVELS = 3
OVERVIEW_RESAMPLING = 'average'

for file in glob.glob("output3/*.tif"):
    output_path = f"output3/{file.split("/")[-1]}"
    
    # Create a temporary file to hold the COG
    with tempfile.NamedTemporaryFile(suffix='.tif', delete=False) as temp_file:       
        # Create COG with overviews and nodata value
        cog_translate(
            file,
            temp_file.name,
            cog_profiles.get("deflate"),
            overview_level=OVERVIEW_LEVELS,
            overview_resampling=OVERVIEW_RESAMPLING,
            nodata=0
        )
        # Move the temporary file to the desired local path
        os.rename(temp_file.name, output_path)

Reading input: output3/GOES-CH4_Mexico_Durango_BV1_BV1-1_2019-04-07T18:26:00Z.tif

Adding overviews...
Updating dataset tags...
Writing output to: /var/folders/6f/r0g105hx7l5g3qw69th62h100000gp/T/tmpd1a7ggu6.tif
Reading input: output3/GOES-CH4_Mexico_Durango_BV1_BV1-1_2019-04-07T18:36:00Z.tif

Adding overviews...
Updating dataset tags...
Writing output to: /var/folders/6f/r0g105hx7l5g3qw69th62h100000gp/T/tmpagktrif4.tif
Reading input: output3/GOES-CH4_Mexico_Durango_BV2_BV2-1_2019-04-09T18:26:00Z.tif

Adding overviews...
Updating dataset tags...
Writing output to: /var/folders/6f/r0g105hx7l5g3qw69th62h100000gp/T/tmp1ymy0puz.tif
Reading input: output3/GOES-CH4_Mexico_Durango_BV2_BV2-1_2019-04-09T18:36:00Z.tif

Adding overviews...
Updating dataset tags...
Writing output to: /var/folders/6f/r0g105hx7l5g3qw69th62h100000gp/T/tmpxevevopk.tif
Reading input: output3/GOES-CH4_Mexico_Durango_BV2_BV2-2_2019-04-10T17:21:00Z.tif

Adding overviews...
Updating dataset tags...
Writing output to: /var/

In [83]:
import pandas as pd
files = glob.glob("output3/*")
data=[]
for file in files:
    file = file.split("/")[-1].split('.')[0]
    data.append([file.split("/")[-1].split('.')[0][:-20], file.split("/")[-1].split('.')[0][-20:]])
df = pd.DataFrame(data, columns=["group_key", "datetime"])
df['datetime'] = pd.to_datetime(df['datetime'])  # Convert to datetime

# Group by 'group_key' and get the minimum date for each group
result = df.groupby('group_key')['datetime'].min().reset_index()
result['datetime'] = result['datetime'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')
result['group_key_datetime'] = result['group_key'] + '' + result['datetime']
filtered_files=list(result['group_key_datetime'])

In [84]:
import rasterio
import geopandas as gpd
from rasterio import features
from shapely.geometry import shape, MultiPolygon
from shapely.ops import unary_union
import numpy as np
import pandas as pd

def raster_to_enclosing_polygon(tiff_file): 
    with rasterio.open(f"output3/{tiff_file}.tif") as src:
        image = src.read(1)  # Read the first band
        image = image.astype(np.float32)

        # Generate shapes for non-zero regions
        mask = image != 0
        shapes = features.shapes(image, mask=mask, transform=src.transform)
        
        # Collect all polygons
        polygons = [shape(geom) for geom, value in shapes if value != 0]
        
        # Merge all polygons into one outer boundary
        multi_polygon = MultiPolygon(polygons)
        enclosing_polygon = unary_union(multi_polygon).convex_hull
        
        return enclosing_polygon, src.crs  # Return the geometry and CRS

def process_rasters_and_save(filtered_files, output_geojson):
    # Create an empty GeoDataFrame
    gdf = gpd.GeoDataFrame(columns=['geometry', 'filename'], crs='EPSG:4326')  # Default CRS
    
    # Create an empty list to collect rows for concatenation
    rows = []

    for file in filtered_files:
        # Get the enclosing polygon and CRS for each raster
        geometry, crs = raster_to_enclosing_polygon(file)
        
        # Create a new row as a dictionary
        new_row = {'geometry': geometry, 'filename': file[:-21]}
        
        # Append the row to the list
        rows.append(new_row)
    
    # Convert the list of rows into a GeoDataFrame
    new_gdf = gpd.GeoDataFrame(rows, columns=['geometry', 'filename'], crs=crs)
    
    # Concatenate the new GeoDataFrame with the existing one
    gdf = pd.concat([gdf, new_gdf], ignore_index=True)
    
    # Save the GeoDataFrame to GeoJSON
    gdf.to_file(output_geojson, driver="GeoJSON")
    print(gdf)
    
    print(f"GeoJSON saved to {output_geojson}")

# Usage
process_rasters_and_save(filtered_files, 'enclosing_shapes.geojson')


                                             geometry  \
0   POLYGON ((-104.49154 26.16844, -104.54644 26.2...   
1   POLYGON ((-104.52474 26.24287, -104.55304 26.2...   
2   POLYGON ((-104.34563 26.02893, -104.37335 26.0...   
3   POLYGON ((-104.33024 26.07158, -104.35852 26.0...   
4   POLYGON ((-104.28103 25.98937, -104.33751 26.0...   
5   POLYGON ((-104.34285 26.07179, -104.35703 26.0...   
6   POLYGON ((-104.3507 26.05286, -104.36471 26.06...   
7   POLYGON ((-103.89216 25.67443, -103.96056 25.7...   
8   POLYGON ((-103.927 25.62727, -103.95394 25.654...   
9   POLYGON ((-103.9494 25.60269, -103.96293 25.64...   
10  POLYGON ((-103.93068 25.64735, -103.95818 25.6...   
11  POLYGON ((-86.90008 40.99919, -86.93046 41.014...   
12  POLYGON ((-87.33267 40.67778, -87.34768 40.692...   
13  POLYGON ((-103.86172 31.5363, -103.86172 31.56...   

                             filename  
0   GOES-CH4_Mexico_Durango_BV1_BV1-1  
1   GOES-CH4_Mexico_Durango_BV1_BV1-2  
2   GOES-CH4_Mexico_Dura